<a href="https://colab.research.google.com/github/AliKarimiENT/Machine_Translation_EN_FA/blob/main/Machine_Translation_EN_FA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Machine Translation Project

# Install libraries

In [1]:
!pip install -U -q PyDrive
!pip install tensorflow
!pip install -U numpy==1.21
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.7 MB 26.6 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Dataset




> The data are located in Google Drive. The TEP.en-fa.en file contains English sentences with their Farsi translation in the TEP.en-fa.fa file. 

Load the English and Farsi data from these files from running the cell below.

In [2]:
import os

def load_data(path):
    """
    Load dataset
    """
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()

    return data.split('\n')

In [9]:
import helper
from keras.layers import GRU , LSTM , Input , Dense , TimeDistributed , Bidirectional
from keras.models import Model
from keras.layers import Activation
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.layers import RepeatVector
from keras.layers.embeddings import Embedding

# Connect to Google Drive to load data
from google.colab import drive
drive.mount('/content/drive')

# Load English data
english_sentences = load_data('/content/drive/MyDrive/University Tehran /TEP.en-fa.en')[0:13000]

# Load Farsi data 
farsi_sentences = load_data('/content/drive/MyDrive/University Tehran /TEP.en-fa.fa')[0:13000]

print('Dataset Loaded')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset Loaded


# Files

Each sentence in TEP.en-fa.en contains an English sentence with respective translation in each line of TEP.en-fa.fa. For example check these two lines of them 

In [10]:
for sample in range(2):
  print('TEP.en-fa.en Line {}:  {}'.format(sample + 1, english_sentences[sample]))
  print('TEP.en-fa.fa Line {}:  {}'.format(sample + 1, farsi_sentences[sample]))

TEP.en-fa.en Line 1:  raspy breathing .
TEP.en-fa.fa Line 1:  صداي خر خر .
TEP.en-fa.en Line 2:  dad .
TEP.en-fa.fa Line 2:  پدر .


# Complexity of the vocabulary

In [11]:
import collections
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
farsi_words_counter = collections.Counter([word for sentence in farsi_sentences for word in sentence.split()])

print('{} English words.'.format(len(english_words_counter)))
print('{} Farsi words.'.format(len(farsi_words_counter)))

7412 English words.
10816 Farsi words.


# Tokenize

In [12]:
from keras.preprocessing.text import Tokenizer

def tokenize(x):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(x)
  sequences = tokenizer.texts_to_sequences(x)
  return sequences,tokenizer

eng_text_tokenized , en_text_tokenizer = tokenize(english_sentences)
print(en_text_tokenizer.word_index)

fa_text_tokenized , fa_text_tokenizer = tokenize(farsi_sentences)
print(fa_text_tokenizer.word_index)

{'you': 1, 'the': 2, 'i': 3, 'to': 4, 'a': 5, 'it': 6, 'and': 7, 'of': 8, 'is': 9, 'that': 10, 'me': 11, 'we': 12, 'in': 13, 'what': 14, 'this': 15, 'on': 16, 'are': 17, 'your': 18, 'for': 19, 'do': 20, 'not': 21, 'have': 22, 'my': 23, 'no': 24, 'be': 25, 'all': 26, 'was': 27, 'dont': 28, 'get': 29, 'he': 30, 'its': 31, 'here': 32, 'know': 33, 'just': 34, 'with': 35, 'im': 36, 'go': 37, 'up': 38, 'right': 39, 'they': 40, 'can': 41, 'him': 42, 'but': 43, 'now': 44, 'there': 45, 'so': 46, 'come': 47, 'out': 48, 'if': 49, 'were': 50, 'got': 51, 'about': 52, 'one': 53, 'youre': 54, 'how': 55, 'thats': 56, 'like': 57, 'will': 58, 'did': 59, 'well': 60, 'see': 61, 'at': 62, 'think': 63, 'them': 64, 'us': 65, 'as': 66, 'good': 67, 'yeah': 68, 'oh': 69, 'back': 70, 'from': 71, 'time': 72, 'want': 73, 'hey': 74, 'going': 75, 'when': 76, 'take': 77, 'why': 78, 'need': 79, 'his': 80, 'down': 81, 'cant': 82, 'gonna': 83, 'hes': 84, 'look': 85, 'an': 86, 'tell': 87, 'okay': 88, 'mr': 89, 'her': 90,

# Padding

When we are combining sequence of words together, each sequence needs to be the same length. We add padding to the end of the sequences to make them the same length.

In [13]:
import numpy as np 
from keras.preprocessing.sequence import pad_sequences

def pad(x, length = None):
  # x is list of sequences
  # length: Length to pad the sequence to. If it is None , use length of longest sequence in x 

  # It will return the padded numpy array of sequences
  if length == None:
    longest_sequence = max(x,key= len)
    return pad_sequences(x,len(longest_sequence),padding = 'post')
  return pad_sequences(x,length , padding= 'post')



# Pad tokenized output
en_padded = pad(eng_text_tokenized)
fa_padded = pad(fa_text_tokenized)


# Preprocess

In [14]:
def preprocess(x,y):
  """
    x: Feature list of sentences
    y: Label list of sentences

    It returns Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
  """

  preprocess_x , x_tk = tokenize(x)
  preprocess_y , y_tk = tokenize(y)

  preprocess_x = pad(preprocess_x)
  preprocess_y = pad(preprocess_y)

  # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
  preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
  return preprocess_x, preprocess_y, x_tk, y_tk
 
preproc_english_sentences,preproc_farsi_sentences,english_tokenizer,farsi_tokenizer = preprocess(english_sentences,farsi_sentences)

Data preprocess done !

In [15]:
print(preproc_english_sentences.shape)
print(preproc_farsi_sentences.shape)

(13000, 34)
(13000, 30, 1)


# 1.  Use **RNN** model 

A basic RNN model is a good baseline for sequence data.
We are going to build a RNN that translates English to

In [16]:
def rnn_model(input_shape,farsi_vocab_size):
  """
    input_shape: Tuple of input shape
    farsi_voca_size: Number of unique 
  """
  learning_rate = 0.01
  inputs = Input(shape=input_shape[1:])
  x = GRU(512,return_sequences=True)(inputs)
  x = TimeDistributed(Dense(farsi_vocab_size,activation='relu'))(x)
  predictions = Activation('softmax')(x)
  model = Model(inputs = inputs,outputs = predictions)
  model.compile(loss= sparse_categorical_crossentropy,
                optimizer= Adam(learning_rate),
                metrics = ['accuracy'])
  return model


# Reshape the input to work with a basic RNN
temp_x = pad(preproc_english_sentences,preproc_farsi_sentences.shape[1])
temp_x = temp_x.reshape((-1,preproc_farsi_sentences.shape[-2],1))

# Train the neural network
simple_rnn_model = rnn_model(
    temp_x.shape,
    # preproc_farsi_sentences.shape[1],
    # len(english_tokenizer.word_index) + 1, # Add 1 because padding introduces 0
    len(farsi_tokenizer.word_index) + 1 # Add 1 because padding introduces 0
)

simple_rnn_model.summary()
simple_rnn_model.fit(temp_x, preproc_farsi_sentences, batch_size=1024, epochs=10, validation_split=0.2)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 30, 1)]           0         
                                                                 
 gru (GRU)                   (None, 30, 512)           791040    
                                                                 
 time_distributed (TimeDistr  (None, 30, 10794)        5537322   
 ibuted)                                                         
                                                                 
 activation (Activation)     (None, 30, 10794)         0         
                                                                 
Total params: 6,328,362
Trainable params: 6,328,362
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
11/11 [==============================] - 12s 471ms/step - loss: 3.9458 - accuracy: 0.7186 - val_loss: 2.04

In [17]:
simple_rnn_model_scores = simple_rnn_model.evaluate(temp_x,preproc_farsi_sentences,verbose=0)
# By setting verbose 0, 1 or 2 you just say how do you want to 'see' the training progress for each epoch.
print("Model Accuracy: %.2f%%" % (simple_rnn_model_scores[1]*100)) 

Model Accuracy: 77.33%


# 2. Use **Embedding** Model

Using words in embedding model. An Embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.